In [1]:
# Install Unsloth and dependencies
!pip install unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-slg_9pqu/unsloth_3a105e5f7b6e46eeb5b613eb79cb65df
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-slg_9pqu/unsloth_3a105e5f7b6e46eeb5b613eb79cb65df
  Resolved https://github.com/unslothai/unsloth.git to commit c1b73fa8836aa7e8b9ee13d748369f8f61e1fac5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
import csv
import pandas as pd
import torch
from tqdm import tqdm
from unsloth import FastLanguageModel
from transformers import logging
import warnings
from sklearn.metrics import accuracy_score, f1_score
from sklearn.exceptions import UndefinedMetricWarning

# ===========================
# Setup
# ===========================

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
BASE_DIR = "/content"  # Override HOME for Colab
DATA_PATH = os.path.join(BASE_DIR, "data", "data_huang_devansh_processed_equal.csv")
RESULTS_DIR = os.path.join(BASE_DIR, "results", "gemma_results")
os.makedirs(RESULTS_DIR, exist_ok=True)

# ===========================
# Suppress Warnings
# ===========================

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
logging.set_verbosity_error()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# ===========================
# Load Data (last 30%)
# ===========================
print(f"Loading data from: {DATA_PATH}")
raw_rows = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        text = (row.get("Content") or "").strip()
        label = (row.get("normalized_label") or row.get("Label") or "").strip()
        if text and label in {"0", "1"}:
            raw_rows.append((text, int(label)))

df = pd.DataFrame(raw_rows, columns=["text", "label"])
print(f"Full dataset size: {len(df)}")

start_idx = int(len(df) * 0.7)
df = df.iloc[start_idx:].reset_index(drop=True)
print(f"Using last 30%: {len(df)} samples")

Loading data from: /content/data/data_huang_devansh_processed_equal.csv
Full dataset size: 249954
Using last 30%: 2500 samples


In [4]:
# ===========================
# Instruction template
# ===========================
base_instruction = (
    "You are a content moderation assistant. Classify each text as Hate '1' or No Hate '0'. Reply only with the number.\n\n"
)

fs_instruction = (
    "You are a content moderation assistant. Classify each text as Hate '1' or No Hate '0'. "
    "Reply only with the number.\n\n"
    "Example 1:\n"
    "Just wanted to let you know you had 2 imposters ( (talk) \"\"\"\n0\n\n"
    "Example 2:\n"
    "thanking you yin... @user tranquil & beautiful friday & #weekend to you 🙂🖐🏽😎\n0\n\n"
    "Example 3:\n"
    "Radiant: What does Userfy mean? What is VfD?\n0\n\n"
    "Example 4:\n"
    "dead c o o n s is a good thing\n1\n\n"
    "Example 5:\n"
    "\"You are a fool. Hispanics are hispanics are hispanic to the bone. ... conservatives fools.\"\n1\n\n"
    "Example 6:\n"
    "\"DEAR GILLIAM, YOU PIECE OF SHIZZ  YOU THINK YOU KNOW POKEMON? ...\"\n1\n\n"
    "Example 7:\n"
    "its been merged, but how to delete the other article? Shall I have a redirect?\n0\n\n"
    "Example 8:\n"
    "LOL that meme you shared was hilarious, thanks for the laugh!\n0\n\n"
    "Now classify:\n"
)


In [5]:
# ===========================
# Load model and tokenizer
# ===========================
model_name = "unsloth/gemma-3-1b-it-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8192,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


==((====))==  Unsloth 2025.6.1: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


In [6]:
# ===========================
# Inference function
# ===========================
import gc

def run_inference(df_subset, instruction, few_shot=False, batch_size=128):
    preds = []
    if few_shot:
        # sample 3 hate and 3 no-hate examples
        hate_ex = df[df.label == 1].head(3)
        no_ex   = df[df.label == 0].head(3)
        examples = pd.concat([hate_ex, no_ex]).sample(frac=1, random_state=42)
        ex_prompt = ""
        for idx, row in enumerate(examples.itertuples(), 1):
            ex_prompt += f"Example {idx}:\n{row.text}\n{row.label}\n"
        prompt_prefix = instruction + ex_prompt + "Now classify:\n"
    else:
        prompt_prefix = instruction

    texts = df_subset.text.tolist()
    num_batches = (len(texts) + batch_size - 1) // batch_size

    for i in tqdm(range(num_batches), desc="Batched inference"):
        batch_texts = texts[i*batch_size:(i+1)*batch_size]
        prompts     = [prompt_prefix + t for t in batch_texts]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        ).to(device)

        with torch.no_grad():
            out_ids = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=5,
                use_cache=True,
            )

        decoded_batch = tokenizer.batch_decode(out_ids, skip_special_tokens=True)
        for decoded in decoded_batch:
            p = decoded.splitlines()[-1].strip()
            preds.append(int(p) if p in {"0", "1"} else None)

        # —— cleanup to free CUDA memory ——
        del inputs, out_ids, decoded_batch, prompts, batch_texts
        torch.cuda.empty_cache()
        gc.collect()

    return preds

In [ ]:
# ===========================
# Run experiments across seeds
# ===========================
seeds = [42, 123, 456, 789, 101112]
for seed in seeds:
    df_seed = df.sample(frac=1, random_state=seed).reset_index(drop=True)

    # Zero‐shot
    zs_preds = run_inference(df_seed, base_instruction, few_shot=False)
    df_seed['pred_zs'] = zs_preds
    eval_zs = df_seed.dropna(subset=['pred_zs'])
    acc_zs = accuracy_score(eval_zs.label, eval_zs.pred_zs)
    f1_zs_h = f1_score(eval_zs.label, eval_zs.pred_zs, pos_label=1)
    f1_zs_n = f1_score(eval_zs.label, eval_zs.pred_zs, pos_label=0)
    pd.DataFrame([{
        'accuracy':  acc_zs,
        'f1_Hate':   f1_zs_h,
        'f1_NoHate': f1_zs_n
    }]).to_csv(
        os.path.join(RESULTS_DIR, f"gemma_base_zs_metrics_seed{seed}.csv"),
        index=False
    )

    # Few‐shot with fixed examples
    fs_preds = run_inference(df_seed, fs_instruction, few_shot=False)
    df_seed['pred_fs'] = fs_preds
    eval_fs = df_seed.dropna(subset=['pred_fs'])
    acc_fs  = accuracy_score(eval_fs.label, eval_fs.pred_fs)
    f1_fs_h = f1_score(eval_fs.label, eval_fs.pred_fs, pos_label=1)
    f1_fs_n = f1_score(eval_fs.label, eval_fs.pred_fs, pos_label=0)
    pd.DataFrame([{
        'accuracy':  acc_fs,
        'f1_Hate':   f1_fs_h,
        'f1_NoHate': f1_fs_n
    }]).to_csv(
        os.path.join(RESULTS_DIR, f"gemma_base_fs_metrics_seed{seed}.csv"),
        index=False
    )

    # free any residual GPU memory between seeds
    torch.cuda.empty_cache()
    gc.collect()

print("Experiments complete.")

Batched inference:  95%|█████████▌| 19/20 [02:23<00:07,  7.65s/it]